In [6]:
# credits to:
# https://www.youtube.com/watch?v=l8ZYCvgGu0o
# https://github.com/Kyubyong/dc_tts
!pip install transformers


     |████████████████████████████████| 1.3MB 11.8MB/s 
     |████████████████████████████████| 2.9MB 58.5MB/s 
     |████████████████████████████████| 1.1MB 55.4MB/s 
     |████████████████████████████████| 890kB 48.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b37b6796f757d2a8085e1bb0e36e2d1ef8b0028913883d8f639777f98a5e440e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Declaration of BERT for answering question

In [7]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import pandas as pd

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
bert_abstract = "Krzysztof January Krawczyk (born September 8, 1946 in Katowice) is a Polish baritone pop singer, guitarist and composer. He was the vocalist of a popular Polish band, Trubadurzy (the Troubadours), from 1963 to 1973 when he started his solo career. He was co-founder of the Post-secondary School of Stage Art in Łódź. His creative activity in the area of music is characterized by a combination of various music genres such as rock and roll, country music and rhythm & blues. His album To co w życiu ważne reached number one on the Polish Music Charts.Krawczyk learned to play the guitar on his own, whereas his vocal abilities were practised when he attended secondary school of music in Łódź. However, he had to quit his musical studies and become an errand-boy, because his father, an actor, died and his mother suffered from depression. He was the only member of the family to earn a living.In 1963 he founded, together with Ryszard Poznakowski, Marian Lichtman, Sławomir Kowalewski and Halina Żytkowiak, one of the best known Polish beat music bands of the 1960s, Trubadurzy, who combined elements of rock with Polish folk music."

def answer_question(question, answer_text):
    input_ids = tokenizer.encode(question, answer_text)
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)

    start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids])) 
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer = tokens[answer_start]
    for i in range(answer_start + 1, answer_end + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]

    return answer

Declaration of Text to Voice

In [1]:
!git clone https://github.com/CorentinJ/Real-Time-Voice-Cloning.git


Cloning into 'Real-Time-Voice-Cloning'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 2540 (delta 0), reused 0 (delta 0), pack-reused 2539
Receiving objects: 100% (2540/2540), 360.90 MiB | 39.06 MiB/s, done.
Resolving deltas: 100% (1404/1404), done.


In [2]:
cd Real-Time-Voice-Cloning/

/content/Real-Time-Voice-Cloning


In [3]:
!pip install -q -r requirements.txt
!apt-get install -qq libportaudio2
!gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
!unzip pretrained.zip

     |████████████████████████████████| 412.3MB 43kB/s 
     |████████████████████████████████| 686kB 47.7MB/s 
     |████████████████████████████████| 184kB 60.6MB/s 
     |████████████████████████████████| 11.6MB 55.9MB/s 
     |████████████████████████████████| 245kB 64.3MB/s 
     |████████████████████████████████| 71.6MB 157kB/s 
     |████████████████████████████████| 3.8MB 50.7MB/s 
     |████████████████████████████████| 512kB 56.1MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 204kB 61.6MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 286kB 63.5MB/s 
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 144628 files 

In [4]:
from google.colab import drive
import os 
drive.mount('/content/drive', force_remount=True)
COLAB = True
print("Note: using Google CoLab")

Mounted at /content/drive
Note: using Google CoLab


In [5]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
AUDIO_PATH = ("/content/drive/My Drive")

encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)
in_fpath = os.path.join(AUDIO_PATH, 'audio.wav') 

reprocessed_wav = encoder.preprocess_wav(in_fpath)
original_wav, sampling_rate = librosa.load(in_fpath)
preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
embed = encoder.embed_utterance(preprocessed_wav)

/content/Real-Time-Voice-Cloning/encoder/audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder/saved_models/pretrained/pretrained.pt


In [23]:
question = "What nationality is Krawczyk?"
text = answer_question(question, bert_abstract)
print(text)

polish


In [24]:
with io.capture_output() as captured:
    specs = synthesizer.synthesize_spectrograms([text], [embed])
generated_wav = vocoder.infer_waveform(specs[0])
generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
display(Audio(generated_wav, rate=synthesizer.sample_rate))

{| ████████████████ 19000/19200 | Batch Size: 2 | Gen Rate: 2.5kHz | }